# Digital Image Processing Project Section A
## Glaucoma Detection
##### Memebrs:
- Syed Qasim Hussain 21i-0379
- Hashir Saeed 21i-0376
- Musfirah 21i-0789

In [11]:
import csv
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, jaccard_score
from keras.models import load_model

# find_optic_disc(image_path):
### This function takes the path of an image as input and attempts to find the optic disc region within the image. Here's how it works:
- It reads the image using cv2.imread() and converts it to grayscale using cv2.cvtColor().
- It applies Gaussian Blur to the grayscale image using cv2.GaussianBlur() to reduce noise.
- It detects edges in the blurred image using the Canny edge detector (cv2.Canny()).
- It finds contours in the edge image using cv2.findContours().
- It loops through each contour and checks if its area is greater than 150 pixels.
- For contours with a large enough area, it calculates the mean intensity within the contour region.
- If the mean intensity is above a certain brightness threshold (120 for TIFF images, 1 for others), it considers the contour as a potential optic disc candidate.
- If any optic disc candidates are found, it selects the largest contour and returns its bounding box coordinates (x, y, x+w, y+h).
- If no optic disc candidates are found, it returns None.

In [2]:
# Function to find optic disc
def find_optic_disc(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    disc_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 150:
            mask = np.zeros_like(gray)
            cv2.drawContours(mask, [contour], 0, 255, -1)
            mean_intensity = cv2.mean(img, mask=mask)[0]

            if image_path.endswith(".tif"):
                brightness_threshold = 120
            else:
                brightness_threshold = 1

            if mean_intensity > brightness_threshold:
                disc_contours.append(contour)

    if disc_contours:
        largest_contour = max(disc_contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        return x, y, x+w, y+h
    else:
        return None

# auto_annotate(data_path):
### This function automatically annotates the optic disc regions in all images (PNG and TIFF) within the specified data_path. It does this by:
- Iterating over all image files in the data_path directory.
- For each image, it calls the find_optic_disc() function to get the optic disc bounding box coordinates.
- If a bounding box is found, it appends a tuple containing the image name and bounding box coordinates to the annotations list.
- After processing all images, it returns the annotations list.

In [3]:
# Function to auto annotate
def auto_annotate(data_path):
    annotations = []
    for image_name in os.listdir(data_path):
        if image_name.endswith((".png", ".tif")):
            image_path = os.path.join(data_path, image_name)
            bbox = find_optic_disc(image_path)
            if bbox is not None:
                annotations.append((image_name, *bbox))
    return annotations

# Function to save annotations to CSV
def save_annotations_to_csv(annotations, output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Filename', 'Xmin', 'Ymin', 'Xmax', 'Ymax'])
        for annotation in annotations:
            writer.writerow(annotation)

### This function creates binary masks for the optic disc regions based on the annotations. It does this by:
- Iterating over each annotation (image name and bounding box coordinates).
- Loading the corresponding image using cv2.imread().
- Creating a blank mask (numpy array of zeros) with the same size as the original image.
- Drawing a filled rectangle on the mask within the bounding box coordinates.
- Resizing the mask to the specified img_size.Appending the resized mask to the masks list.Finally, it returns a numpy array of all masks, with values scaled between 0 and 1.


In [4]:
def create_masks(data_path, annotations, img_size=(256, 256)):
    masks = []
    for annotation in annotations:
        image_name, xmin, ymin, xmax, ymax = annotation
        img_path = os.path.join(data_path, image_name)
        mask = np.zeros((1376, 1376), dtype=np.uint8)
        xmin, ymin, xmax, ymax = map(int, [xmin, ymin, xmax, ymax])
        mask[ymin:ymax, xmin:xmax] = 255
        mask_resized = cv2.resize(mask, img_size)
        masks.append(mask_resized)
    return np.array(masks) / 255.0

In [5]:
def load_images_and_labels(data_path, annotations, img_size=(256, 256)):
    images = []
    masks = create_masks(data_path, annotations, img_size)
    for annotation in annotations:
        image_name, _, _, _, _ = annotation
        img_path = os.path.join(data_path, image_name)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, img_size)
        images.append(img_resized)
    images = np.array(images) / 255.0
    masks = np.expand_dims(masks, axis=-1)
    return images, masks


### This function loads the images and their corresponding masks (labels) based on the annotations. It does this by:
- Calling create_masks() to generate the masks.
- Iterating over each annotation (image name and bounding box coordinates).
- Loading the corresponding image using cv2.imread().
- Resizing the image to the specified img_size.
- Appending the resized image to the images list.
- Converting the images list to a numpy array and scaling the pixel values between 0 and 1.
- Adding an extra channel dimension to the masks array.
- Returning the images and masks arrays.

In [6]:
images_dir = 'C:/Users/DELL/Desktop/RIGA 200 Images'

# Load annotations
annotations = auto_annotate(images_dir)
save_annotations_to_csv(annotations, 'annotations.csv')

# Shuffle and split dataset
annotations_df = pd.read_csv('annotations.csv').sample(frac=1, random_state=42)
train_annotations, test_annotations = train_test_split(annotations_df.values, test_size=0.2, random_state=42)
train_annotations, val_annotations = train_test_split(train_annotations, test_size=0.2, random_state=42)

# Load images and labels
train_images, train_masks = load_images_and_labels(images_dir, train_annotations)
val_images, val_masks = load_images_and_labels(images_dir, val_annotations)
test_images, test_masks = load_images_and_labels(images_dir, test_annotations)


### UNET Model
- This function defines the architecture of the U-Net model for segmentation tasks. It uses convolutional layers, max-pooling layers, and transposed convolutional layers (for upsampling) to create the encoder and decoder parts of the U-Net. The model takes an input image of size input_size and outputs a binary mask of the same spatial dimensions.

In [7]:
def unet_model(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Dropout(0.1)(c1)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Dropout(0.1)(c2)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Dropout(0.2)(c3)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Dropout(0.2)(c4)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Dropout(0.3)(c5)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Dropout(0.2)(c6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Dropout(0.2)(c7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Dropout(0.1)(c8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Dropout(0.1)(c9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

#### train_model(model, train_images, train_labels, val_images, val_labels):
- This function trains the provided model using the training data (train_images and train_labels) and validation data (val_images and val_labels). It does this by:
- Compiling the model with the Adam optimizer and binary cross-entropy loss.
- Creating data generators for the training and validation data using ImageDataGenerator().
- Fitting the model to the training data generator for 50 epochs, using the validation data generator for validation.
- Returning the training history.

In [8]:
# Train model function
def train_model(model, train_images, train_labels, val_images, val_labels):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    datagen = ImageDataGenerator()
    train_gen = datagen.flow(train_images, train_labels, shuffle=True)
    val_gen = datagen.flow(val_images, val_labels, shuffle=True)

    history = model.fit(train_gen, epochs=50, validation_data=val_gen)
    return history


In [36]:
# Initialize and train U-Net model for optic disc segmentation
od_model = unet_model(input_size=(256, 256, 3))
history_od = train_model(od_model, train_images, train_masks, val_images, val_masks)


Epoch 1/50


C:\Users\DELL\Desktop\proj_1\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 171s 41s/step - accuracy: 0.6589 - loss: 0.6918 - val_accuracy: 0.7005 - val_loss: 0.6687
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 153s 39s/step - accuracy: 0.7009 - loss: 0.6175 - val_accuracy: 0.6988 - val_loss: 1.8267
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 141s 35s/step - accuracy: 0.7585 - loss: 1.3112 - val_accuracy: 0.7693 - val_loss: 0.6278
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 142s 36s/step - accuracy: 0.7201 - loss: 0.6440 - val_accuracy: 0.7695 - val_loss: 0.6304
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 140s 35s/step - accuracy: 0.7640 - loss: 0.6289 - val_accuracy: 0.7728 - val_loss: 0.6112
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 139s 35s/step - accuracy: 0.7279 - loss: 0.6128 - val_accuracy: 0.7443 - val_loss: 0.5322
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 136s 35s/step - accuracy: 0.7278 - loss: 0.5183 - val_accuracy: 0.7687 - val_loss: 0.4507
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 145s 36s/step - accuracy: 0.7578 - loss: 0.4661 - val_accuracy: 0.7769 - val_loss: 0.4358
Epo

In [32]:
def evaluate_model(model, test_images, test_labels):
    results = model.evaluate(test_images, test_labels)
    print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

evaluate_model(od_model, test_images, test_masks)

2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.8227 - loss: 0.2762
Test Loss: 0.2747751772403717, Test Accuracy: 0.8222770690917969


#### refine_oc_segmentation(od_image, oc_model):
- This function takes an optic disc image (od_image) and the optic cup model (oc_model) as input. It refines the optic cup segmentation by:
- Predicting the optic cup mask using the optic cup model (oc_model.predict()).
- Returning the predicted optic cup mask.

In [15]:
# Optic cup segmentation refinement function
def refine_oc_segmentation(od_image, oc_model):
    oc_mask = oc_model.predict(np.expand_dims(od_image, axis=0))[0]
    return oc_mask

In [16]:
 #Load images and labels for optic cup
train_images_oc, train_masks_oc = load_images_and_labels(images_dir, train_annotations)
test_images_oc, test_masks_oc = load_images_and_labels(images_dir, test_annotations)

In [41]:
# Define U-Net model for Optic Cup
oc_model = unet_model(input_size=(256, 256, 3))

# Train OC model
history_oc = train_model(oc_model, train_images_oc, train_masks_oc, val_images, val_masks)


Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 164s 39s/step - accuracy: 0.6701 - loss: 0.6911 - val_accuracy: 0.7694 - val_loss: 0.6707
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 150s 38s/step - accuracy: 0.7549 - loss: 0.6575 - val_accuracy: 0.7696 - val_loss: 0.5771
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 148s 38s/step - accuracy: 0.7218 - loss: 0.6195 - val_accuracy: 0.7694 - val_loss: 0.5663
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 148s 37s/step - accuracy: 0.7038 - loss: 0.8309 - val_accuracy: 0.7696 - val_loss: 0.5557
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 139s 35s/step - accuracy: 0.7485 - loss: 0.5858 - val_accuracy: 0.7760 - val_loss: 0.6027
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 149s 38s/step - accuracy: 0.7536 - loss: 0.6088 - val_accuracy: 0.7524 - val_loss: 0.5623
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 150s 38s/step - accuracy: 0.7097 - loss: 0.8033 - val_accuracy: 0.7862 - val_loss: 0.5937
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 155s 39s/step - accuracy: 0.7459 - loss: 0.6154 - val_accuracy: 0.7709 - val_loss:

In [17]:
# Evaluate OC model
evaluate_model(oc_model, test_images_oc, test_masks_oc)

2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.8279 - loss: 0.2780
Test Loss: 0.2767447829246521, Test Accuracy: 0.827144980430603


In [67]:
# Save the trained OD model in the native Keras format
od_model.save('od_model.keras')

In [68]:
# Save the trained OC model in the native Keras format
oc_model.save('oc_model.keras')

In [12]:
# # Load the saved OD model
# od_model = load_model('od_model.keras', compile=False)

# # Re-initialize the optimizer
# od_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Load the saved OD model
od_model = load_model('od_model.keras')


C:\Users\DELL\Desktop\proj_1\env\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 48 variables whereas the saved optimizer has 94 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [13]:
# Load the saved OC model
oc_model = load_model('oc_model.keras')

### integrate_cdr_calculation(image_path, od_model, oc_model):
#### This function integrates the optic disc and optic cup segmentation models to calculate the CDR for a given image. It does this by:
- Loading the image using cv2.imread() and preprocessing it (resizing and scaling).
- Predicting the optic disc mask using the optic disc model (od_model.predict()).
- Refining the optic cup segmentation using the refine_oc_segmentation() function and the optic cup model (oc_model).
- Calculating the CDR using the calculate_cdr() function with the optic disc and optic cup masks.
- Returning the calculated CDR value.

### calculate_cdr(od_mask, oc_mask):
#### This function calculates the Cup-to-Disc Ratio (CDR) given the optic disc mask (od_mask) and optic cup mask (oc_mask). It does this by:
- Counting the number of pixels above a threshold (0.5) in the optic disc mask to get the optic disc area (od_area).
- Counting the number of pixels above a threshold (0.5) in the optic cup mask to get the optic cup area (oc_area).
- Calculating the CDR by dividing the optic cup area by the optic disc area (oc_area / od_area).
- If the optic disc area is 0, it returns 0 as the CDR.


In [18]:
# Calculate CDR
def calculate_cdr(od_mask, oc_mask):
    od_area = np.sum(od_mask > 0.5)
    oc_area = np.sum(oc_mask > 0.5)
    cdr = oc_area / od_area if od_area > 0 else 0
    return cdr

def integrate_cdr_calculation(image_path, od_model, oc_model):
    img = cv2.imread(image_path)
    img_preprocessed = cv2.resize(img, (256, 256)) / 255.0

    od_mask = od_model.predict(np.expand_dims(img_preprocessed, axis=0))[0]
    oc_mask = refine_oc_segmentation(img_preprocessed, oc_model)

    cdr = calculate_cdr(od_mask, oc_mask)
    return cdr

In [29]:
# Example usage for a sample image
sample_image_path = 'C:/Users/DELL/Desktop/RIGA 200 Images/49.png'
cdr_value = integrate_cdr_calculation(sample_image_path, od_model, oc_model)
print(f"Cup-to-Disc Ratio (CDR) for sample image: {cdr_value}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 809ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 765ms/step
Cup-to-Disc Ratio (CDR) for sample image: 0.697761388739107


### evaluate_segmentation_performance(y_true, y_pred):
### This function evaluates the segmentation performance of a model by calculating various metrics (accuracy, precision, recall, and IoU) given the true labels (y_true) and predicted masks (y_pred). It does this by:
- Flattening the true labels and predicted masks, and converting them to binary arrays using a threshold of 0.5.
- Calculating accuracy using accuracy_score() from sklearn.metrics.
- Calculating precision using precision_score() from sklearn.metrics.
- Calculating recall using recall_score() from sklearn.metrics.
- Calculating IoU (Intersection over Union) using jaccard_score() from sklearn.metrics.
- Returning the calculated metrics.

In [20]:
def evaluate_segmentation_performance(y_true, y_pred):
    y_true_flat = (y_true.flatten() > 0.5).astype(int)
    y_pred_flat = (y_pred.flatten() > 0.5).astype(int)
    accuracy = accuracy_score(y_true_flat, y_pred_flat)
    precision = precision_score(y_true_flat, y_pred_flat)
    recall = recall_score(y_true_flat, y_pred_flat)
    iou = jaccard_score(y_true_flat, y_pred_flat)
    return accuracy, precision, recall, iou


In [30]:
# Evaluate OD model performance
od_accuracy, od_precision, od_recall, od_iou = evaluate_segmentation_performance(test_masks, od_model.predict(test_images))
print(f"OD Segmentation - Accuracy: {od_accuracy}, Precision: {od_precision}, Recall: {od_recall}, IoU: {od_iou}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step
OD Segmentation - Accuracy: 0.8224220275878906, Precision: 0.5672413741795266, Recall: 0.9513494808537257, IoU: 0.5512507832457705


In [31]:
# Evaluate OC model performance
oc_accuracy, oc_precision, oc_recall, oc_iou = evaluate_segmentation_performance(test_masks_oc, oc_model.predict(test_images_oc))
print(f"OC Segmentation - Accuracy: {oc_accuracy}, Precision: {oc_precision}, Recall: {oc_recall}, IoU: {oc_iou}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step
OC Segmentation - Accuracy: 0.8272994995117188, Precision: 0.5773617766838919, Recall: 0.9210242179532611, IoU: 0.5501263991350801
